In [ ]:
!unzip \*.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
data = pd.read_csv("train.csv")
data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# Vemos las variables
features = data.columns.values
features

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [ ]:
# Info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# Vemos que columnas tienen valores vacios y cuantos
dict_nuls = {}
for col in features:
  if data[col].isnull().any():
    dict_nuls[col] = data[col].isnull().sum()
dict_nuls

{'Age': 177, 'Cabin': 687, 'Embarked': 2}

In [ ]:
# Vemos cuantos valores distintos hay en ticket
data['Ticket'].nunique() #681, 70% son distintos
# Eliminamos
del data['Ticket']
# Eliminamos embarked oporuqe no deberia influir hacia donde iba
#del data['Embarked']

In [ ]:
# El nombre no deberia influir (igual en vola si es cuico sobrevive jajaj)
del data['Name']

# El sexo se debe hacer dummie
# La edad llenar con el promedio
# CAbina llenamos con None y dummie


In [ ]:
from sklearn.preprocessing import LabelEncoder
del data['PassengerId']
data.dropna(axis =0, subset = ['Embarked'], inplace = True)
X = data.iloc[:,1:]
y = data.iloc[:,0]
encoder = LabelEncoder()
X['Sex'] = encoder.fit_transform(X['Sex'])


GRAFICA PO MADAFACA

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
categorical = ['Sex','Embarked']
numerical = ['Age','Pclass','SibSp','Parch','Fare']
features = categorical + numerical
X = X[features]
X

,Sex,Embarked,Age,Pclass,SibSp,Parch,Fare
0,1,S,22.0,3,1,0,7.2500
1,0,C,38.0,1,1,0,71.2833
2,0,S,26.0,3,0,0,7.9250
3,0,S,35.0,1,1,0,53.1000
4,1,S,35.0,3,0,0,8.0500
...,...,...,...,...,...,...,...
886,1,S,27.0,2,0,0,13.0000
887,0,S,19.0,1,0,0,30.0000
888,0,S,NaN,3,1,2,23.4500
889,1,C,26.0,1,0,0,30.0000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X,y)

cat_pipe = Pipeline([
                     ('encoder', OrdinalEncoder())
])
num_pipe = Pipeline([
                     ('imputer', SimpleImputer(strategy = 'median'))
])
preprocess = ColumnTransformer([
                                ('num', num_pipe,numerical),
                                ('cat', cat_pipe, categorical)
])

pipe = Pipeline([
                 ('preprocess', preprocess),
                 ('classifier', RandomForestClassifier())
])

#pipe.fit(X_train, y_train)


## GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
params =  {
    'classifier__n_estimators' : (10,100,500,1000),
    'classifier__max_depth'    : (3,5,10,15)
}

In [ ]:
grid = GridSearchCV(pipe, params, scoring='accuracy', cv = 3)
grid.fit(X,y)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocess',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                      

In [ ]:
list(zip(X.columns.values,grid.best_estimator_.steps[1][1].feature_importances_))


[('Sex', 0.20792442384552706),
 ('Embarked', 0.09893044943341404),
 ('Age', 0.05635119388039362),
 ('Pclass', 0.042725170283419704),
 ('SibSp', 0.24003064947666314),
 ('Parch', 0.31759019589010024),
 ('Fare', 0.0364479171904821)]

In [ ]:
grid.best_score_

0.8223033639700307

## Test

In [ ]:
df_test = pd.read_csv('test.csv')
index_test = df_test['PassengerId']
df_test = df_test[features]
df_test['Sex'] = encoder.transform(df_test['Sex'])
preds = grid.predict(df_test)

# To Kaggle

In [ ]:
from google.colab import files
sub = pd.DataFrame({'PassengerId': index_test, 'Survived': preds})
sub.set_index('PassengerId', inplace=  True)
sub.to_csv('submission.csv')
files.download('submission.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>